In [1]:
import requests
import pprint
import pandas as pd
import io

In [2]:
def get_data(country, country_pop, province=""):
    """Retrieves data from API, returns cleaned DF 
    
    Country is inputted from user. I'm just going a simple Google 
        search to populate this.
    
    """
    url = f'https://api.covid19api.com/dayone/country/{country}/status/confirmed/live'
    j = requests.get(url).json()

    raw_df = pd.DataFrame.from_dict(j)
    
    df = raw_df.copy()
    
    df = df[df["Province"] == province]
    
    df = df.reset_index()
    
    df['Date'] = df['Date'].astype('datetime64[ns]')
    df["Daily Cases"] = 0
    
    for i in range(1, len(df)):
        df.loc[i, 'Daily Cases'] = df.loc[i, 'Cases'] - df.loc[i-1, 'Cases']
        
    df = df[df['Daily Cases'] >= 0]
    
    df['DCRA'] = df['Daily Cases'].rolling(7).mean()  # Daily Cases Rolling Average
    
    df['DCRA Per Capita'] = df['DCRA']/country_pop * 100000
    
    return df

In [3]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Legend, HoverTool
from bokeh.palettes import all_palettes
output_notebook()

Loading BokehJS ...

In [4]:
# Don't be intimated by the below, it's just an alternative object instantiation
#     using dataclasses. See https://docs.python.org/3/library/dataclasses.html
#     and https://www.infoworld.com/article/3563878/how-to-use-python-dataclasses.html
#     for more details

from dataclasses import dataclass, field

@dataclass
class InputConfig:
    country: str
    base_pop: int
    province: str = ""
    df: pd.DataFrame = field(init=False)
    data_src: ColumnDataSource = field(init=False)
        
    def __post_init__(self):
        self.df = get_data(self.country, self.base_pop, province=self.province)
        self.data_src = ColumnDataSource(self.df)

In [5]:
# Below is equivalent to the above, but in a more recognizable form

# class InputConfig:
#     """ Object that holds data and labels of data for graphing """
    
#     def __init__(self, country, base_pop, province=""):
        
#         self.country = country
#         self.base_pop = base_pop
#         self.province = province
#         self.df = get_data(country, base_pop, province=province)
#         self.data_src = ColumnDataSource(self.df)

In [6]:
def get_countries(show_results=True):
    """ Utility that gets country names. Useful for populating InputConfig.country above """
    url = f'https://api.covid19api.com/countries'
    result = requests.get(url).json()
    if show_results:
        pp = pprint.PrettyPrinter(indent=4)
        pp.pprint(result)
    
    return result

In [7]:
def get_hover_tool(renderer):

    return HoverTool(
        renderers=[renderer],
        tooltips=[
            ("Daily New Cases 7-Day Rolling Avg", "@{DCRA Per Capita} per 100k ppl"),
            ("Daily New Cases Raw Count", "@{Daily Cases} Total"),
            ("Country", "@{Country}"),
            ("Date", "@Date{%F}"),
        ],
        formatters={
            '@Date': 'datetime',
        }
    )

def format_country_label(input_config):
    legend_label = f'{input_config.country.title().replace("-", " ")}'
    if input_config.province:
        legend_label = f"{legend_label} ({input_config.province.title()})"
    return legend_label

In [8]:
def graph_data(input_configs):
    p = figure(
        plot_width=800,
        plot_height=400,
        x_axis_type='datetime',
        x_axis_label='Date (M/YYYY)',
        y_axis_label='Daily New Cases per 100 000 people',
    )
    
    legend = Legend()
    legend.click_policy="hide"
    p.add_layout(legend, 'right')
    
    colors = all_palettes['Colorblind'][3]
    if len(input_configs) > 3:
        colors = all_palettes['Colorblind'][len(input_configs)]

    for input_config, color in zip(input_configs, colors):

        curr_series = p.circle(
            x="Date",
            y="DCRA Per Capita",
            fill_color=color,
            line_color=color,
            fill_alpha=0.8,
            legend_label=format_country_label(input_config),
            source=input_config.data_src
        )
        
        p.add_tools(get_hover_tool(curr_series))

    show(p)

In [9]:
lockedin = [
    InputConfig("germany", 83020000),
    InputConfig("france", 66990000),
    InputConfig("italy", 60360000),
    InputConfig("portugal", 10280000),
    InputConfig("spain", 46940000),
    InputConfig("canada", 14750000, province="Ontario"),
    InputConfig("greece", 10720000),
]

In [10]:
nah_configs = [
    InputConfig("spain", 46940000),
    InputConfig("france", 66990000),
    InputConfig("bosnia-and-herzegovina", 1845000),
    InputConfig("czech-republic", 10690000),
    InputConfig("netherlands", 17280000),
    InputConfig("croatia", 4058000),
    InputConfig("romania", 19410000),
]

In [11]:
maybe_configs = [
    InputConfig("germany", 83020000),
    InputConfig("italy", 60360000),
    InputConfig("canada", 14750000, province="Ontario"),
    InputConfig("portugal", 10280000),
    InputConfig("kosovo", 10280000),
    InputConfig("albania", 2846000),
    InputConfig("belarus", 9485000),
    InputConfig("poland", 38000000),
]

In [12]:
new_configs = [
    InputConfig("bulgaria", 7000000),
    InputConfig("cyprus", 1189000),
    InputConfig("estonia", 1329000),
    InputConfig("greece", 10720000),
    InputConfig("latvia", 1920000),
    InputConfig("slovenia", 2081000),
    InputConfig("serbia", 6982000),
    InputConfig("switzerland", 8570000),   
]

In [13]:
new_configs = [
    InputConfig("ukraine", 42000000),
    InputConfig("ireland", 5000000),
    InputConfig("sweden", 10230000),

]

In [14]:
us_config = InputConfig("united-states", 328200000)

In [10]:
input_configs.append(us_config)

NameError: name 'input_configs' is not defined

In [10]:
graph_data(lockedin)

In [65]:
graph_data(nah_configs)

In [42]:
def get_europian_slugs():
    europe_iso2 = [
        'AL', 'AD', 'AM', 'AT', 'BY', 'BE', 'BA', 'BG', 'CH', 'CY', 'CZ', 'DE',
        'DK', 'EE', 'ES', 'FO', 'FI', 'FR', 'GB', 'GE', 'GI', 'GR', 'HU', 'HR',
        'IE', 'IS', 'IT', 'LI', 'LT', 'LU', 'LV', 'MC', 'MK', 'MT', 'NO', 'NL', 'PL',
        'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'SM', 'TR', 'UA', 'VA',
    ]
    result = []
    
    for country_dict in get_countries():
        if country_dict['ISO2'] in europe_iso2:
            result.append(country_dict['Slug'])
    
    return result

In [43]:
slugs = get_europian_slugs()

[   {'Country': 'Kiribati', 'ISO2': 'KI', 'Slug': 'kiribati'},
    {'Country': 'Luxembourg', 'ISO2': 'LU', 'Slug': 'luxembourg'},
    {'Country': 'Mauritania', 'ISO2': 'MR', 'Slug': 'mauritania'},
    {'Country': 'Mauritius', 'ISO2': 'MU', 'Slug': 'mauritius'},
    {'Country': 'Panama', 'ISO2': 'PA', 'Slug': 'panama'},
    {   'Country': 'Venezuela (Bolivarian Republic)',
        'ISO2': 'VE',
        'Slug': 'venezuela'},
    {'Country': 'French Polynesia', 'ISO2': 'PF', 'Slug': 'french-polynesia'},
    {'Country': 'Kyrgyzstan', 'ISO2': 'KG', 'Slug': 'kyrgyzstan'},
    {'Country': 'Montserrat', 'ISO2': 'MS', 'Slug': 'montserrat'},
    {'Country': 'Nicaragua', 'ISO2': 'NI', 'Slug': 'nicaragua'},
    {'Country': 'Norway', 'ISO2': 'NO', 'Slug': 'norway'},
    {'Country': 'Turkmenistan', 'ISO2': 'TM', 'Slug': 'turkmenistan'},
    {'Country': 'Argentina', 'ISO2': 'AR', 'Slug': 'argentina'},
    {'Country': 'Malaysia', 'ISO2': 'MY', 'Slug': 'malaysia'},
    {'Country': 'Pakistan', 'ISO2': '

In [44]:
slugs

['luxembourg',
 'norway',
 'bulgaria',
 'gibraltar',
 'sweden',
 'monaco',
 'andorra',
 'italy',
 'switzerland',
 'georgia',
 'portugal',
 'spain',
 'iceland',
 'belarus',
 'turkey',
 'belgium',
 'france',
 'liechtenstein',
 'czech-republic',
 'netherlands',
 'austria',
 'hungary',
 'slovenia',
 'finland',
 'macedonia',
 'san-marino',
 'germany',
 'holy-see-vatican-city-state',
 'russia',
 'lithuania',
 'cyprus',
 'croatia',
 'armenia',
 'albania',
 'faroe-islands',
 'ukraine',
 'greece',
 'malta',
 'ireland',
 'romania',
 'latvia',
 'poland',
 'denmark',
 'bosnia-and-herzegovina',
 'united-kingdom',
 'estonia',
 'slovakia']

In [ ]:
EUROPE_ISO2 = [
    'AL', 'AD', 'AM', 'AT', 'BY', 'BE', 'BA', 'BG', 'CH', 'CY', 'CZ', 'DE',
    'DK', 'EE', 'ES', 'FO', 'FI', 'FR', 'GB', 'GE', 'GI', 'GR', 'HU', 'HR',
    'IE', 'IS', 'IT', 'LI', 'LT', 'LU', 'LV', 'MC', 'MK', 'MT', 'NO', 'NL', 'PL',
    'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'SM', 'TR', 'UA', 'VA',
]